# Elastic constants

For the source code, see [elastic](https://workgraph-collections.readthedocs.io/en/latest/ase/espresso/module.html#workgraph_collections.ase.espresso.elastic.elastic_workgraph).

## Example: Aluminum


### Prepare the inputs and submit the workflow



In [6]:
from ase.build import bulk
from aiida import load_profile
from workgraph_collections.ase.espresso.elastic import ElasticWorkgraph
from aiida_pythonjob.config import config
# allow pickle
config["allow_pickle"] = True

load_profile()

# create input structure
atoms = bulk("Al")


metadata = {
    "options": {
        'prepend_text' : """
eval "$(conda shell.posix hook)"
conda activate aiida
export OMP_NUM_THREADS=1
""",
    }
}

pseudopotentials = {"Al": "Al.pbe-nl-rrkjus_psl.1.0.0.UPF"}
pseudo_dir = "/home/xing/data/ase/espresso_pseudo"
# pseudo_dir = "/home/wang_x3/datas/pseudos/psl"

# we need use high conv_thr to get accurate elastic constants
input_data = {
    "control": {"calculation": "relax", 
                "tstress": True,
                "tprnfor": True,
                "verbosity": "high",
                "etot_conv_thr": 1e-5,
                "forc_conv_thr": 1.0e-5,
                },
    "system": {"ecutwfc": 40, "ecutrho": 320,
               "occupations": "smearing",
               "smearing": "gaussian",
                "degauss": 0.01,
                },
    "electrons": {"conv_thr": 1e-11}
}
#------------------------- Set the inputs -------------------------
wg = ElasticWorkgraph.build_graph(atoms=atoms,
                   computer="localhost",
                   command="mpirun -np 4 pw.x",
                   pseudopotentials=pseudopotentials,
                   pseudo_dir=pseudo_dir,
                   input_data=input_data,
                   kpts=[25, 25, 25],
                   metadata=metadata)
wg.to_html()


Run the calculation

In [2]:
wg.run()

09/04/2025 07:04:36 AM <526049> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [179694|WorkGraphEngine|continue_workgraph]: tasks ready to run: pw_calculator
REPORT:aiida.orm.nodes.process.workflow.workchain.WorkChainNode:[179694|WorkGraphEngine|continue_workgraph]: tasks ready to run: pw_calculator
09/04/2025 07:04:38 AM <526049> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [179694|WorkGraphEngine|on_wait]: Process status: Waiting for child processes: 179698
REPORT:aiida.orm.nodes.process.workflow.workchain.WorkChainNode:[179694|WorkGraphEngine|on_wait]: Process status: Waiting for child processes: 179698
09/04/2025 07:04:41 AM <526049> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [179694|WorkGraphEngine|update_task_state]: Task: pw_calculator, type: PYTHONJOB, finished.
REPORT:aiida.orm.nodes.process.workflow.workchain.WorkChainNode:[179694|WorkGraphEngine|update_task_state]: Task: pw_calculator, type: PYTHONJOB, finishe

In [5]:
#------------------------- Print the output -------------------------
import numpy as np
elastic_constants = wg.outputs.result.value.value
print("Elastic constants: ")
print(np.round(elastic_constants, 2))


Elastic constants: 
[[-109.87  -61.3   -61.3    -0.      4.25   -4.25]
 [ -61.3  -109.87  -61.3    -4.25    0.      4.25]
 [ -61.3   -61.3  -109.87    4.25   -4.25    0.  ]
 [  -0.     -0.     -0.    -38.51   -0.     -0.  ]
 [  -0.     -0.     -0.      0.    -38.51   -0.  ]
 [   0.      0.      0.     -0.     -0.    -38.51]]


The calcualted elastic constants of aluminum are C<sub>11</sub>=117.3 GPa, C<sub>12</sub>=57.6 GPa, and C<sub>44</sub>=38.1 GPa, compared to the values reported by [experiment](https://pubs.aip.org/aip/jap/article/35/6/1825/133322/Elastic-Constants-of-Aluminum): C<sub>11</sub>=116.3 GPa, C<sub>12</sub>=64.8 GPa, and C<sub>44</sub>=30.9 GPa.